## **Import Libraries**

In [1]:
import numpy as np
import requests
import pandas as pd
from io import StringIO
import warnings
warnings.filterwarnings('ignore')
import random
random.seed(6)
!pip install catboost
import catboost 
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.model_selection import KFold,GroupKFold, RepeatedKFold, GridSearchCV, cross_validate, train_test_split
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, roc_auc_score, log_loss
from numpy import mean, std
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
pd.set_option('display.max_colwidth', 250)
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None 
from sklearn.utils import shuffle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.4 MB/s 


# Import Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')
ss = pd.read_csv('/content/drive/My Drive/LADUMA/SampleSubmission.csv')
tdf = pd.read_csv('/content/drive/My Drive/LADUMA/Test.csv')
df = pd.read_csv('/content/drive/My Drive/LADUMA/Train.csv')

Mounted at /content/drive


In [3]:
col=['Game_ID', 'Player_ID', 'id', 'X', 'Y', 'Team', 'Half', 'Manager',
       'Opposition_Team', 'Shots', 'SoT', 'Accurate passes',
       'Inaccurate passes', 'Passes', 'Start_minutes', 'End_minutes',
       'next_player', 'next_x', 'next_y', 'event_id', 'next_team',
       'next_event_id', 'xt_value']

In [4]:
stat1 = pd.read_csv('/content/drive/My Drive/LADUMA/train_game_statistics.csv')

In [5]:
stat2 = pd.read_csv('/content/drive/My Drive/LADUMA/test_game_statistics.csv')

In [6]:
stats=stat1[col].append(stat2[col])
stats=stats.drop_duplicates(['Game_ID','Player_ID','id'])

**Feature Engineering**

In [7]:
teams=list(stats['Team'].unique())
print(len(teams))
teams

22


['Circinus',
 'Backward',
 'Sunflower',
 'Coma Pinwheel',
 'Eye of Sauron',
 'Butterfly',
 'Sombrero',
 'Cosmos Redshift 7',
 'Cigar',
 'Cartwheel',
 'Fireworks',
 'Milky Way',
 'Sculptor',
 'Triangulum',
 'Andromeda',
 'Antennae',
 'Medusa Merger',
 'Comet',
 'Tadpole',
 'Whirlpool',
 'Malin 1',
 'Needle']

In [8]:
teamEncoder = LabelEncoder()
teamEncoder.fit(teams)

LabelEncoder()

In [9]:
stats['Team'] = teamEncoder.transform(stats['Team']).astype('int')


In [10]:
stats['no_of_managers']=stats.groupby('Team')['Manager'].transform('nunique')
stats['no_of_teams']=stats.groupby('Manager')['Team'].transform('nunique')

le=LabelEncoder()
stats.Manager=le.fit_transform(stats.Manager).astype('float')
stats

,Game_ID,Player_ID,id,X,Y,Team,Half,Manager,Opposition_Team,Shots,SoT,Accurate passes,Inaccurate passes,Passes,Start_minutes,End_minutes,next_player,next_x,next_y,event_id,next_team,next_event_id,xt_value,no_of_managers,no_of_teams
0,ID_ZZKYYPVY,Player_O7HO1NOB,1.0,52.5,34.0,6,1st half,52.0,Backward,0.0,0.0,1.0,0.0,1.0,0.03,0.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,3.0
1,ID_ZZKYYPVY,Player_O7HO1NOB,2.0,52.5,34.0,6,1st half,52.0,Backward,0.0,0.0,0.0,0.0,0.0,0.03,0.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,3.0
2,ID_ZZKYYPVY,Player_J0V7B0YE,3.0,37.0,32.7,6,1st half,52.0,Backward,0.0,0.0,0.0,0.0,0.0,0.05,0.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,3.0
3,ID_ZZKYYPVY,Player_J0V7B0YE,4.0,39.5,33.0,6,1st half,52.0,Backward,0.0,0.0,0.0,0.0,0.0,0.07,0.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,3.0
4,ID_ZZKYYPVY,Player_J0V7B0YE,5.0,39.5,33.0,6,1st half,52.0,Backward,0.0,0.0,0.0,1.0,1.0,0.07,0.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802093,ID_03VRW8WV,Player_Y6ZZ2X44,3661.0,42.1,57.4,1,2nd half,23.0,Comet,0.0,0.0,0.0,0.0,0.0,96.33,96.67,Player_Y6ZZ2X44,42.1,57.4,2041.0,Antennae,2041.0,0.0,6,2.0
802094,ID_03VRW8WV,Player_Y6ZZ2X44,3662.0,42.1,57.4,1,2nd half,23.0,Comet,0.0,0.0,0.0,0.0,0.0,96.33,96.67,Player_Y6ZZ2X44,42.1,57.4,2041.0,Antennae,2041.0,0.0,6,2.0
802095,ID_03VRW8WV,Player_6DVBNW9L,3663.0,7.5,19.6,8,2nd half,26.0,Antennae,0.0,0.0,0.0,0.0,0.0,96.42,96.75,Player_6DVBNW9L,7.5,19.6,2042.0,Comet,2042.0,0.0,3,2.0
802096,ID_03VRW8WV,Player_6DVBNW9L,3664.0,7.5,19.6,8,2nd half,26.0,Antennae,0.0,0.0,0.0,0.0,0.0,96.42,96.75,Player_77KCLG39,23.0,0.0,2042.0,Comet,2043.0,0.0,3,2.0


In [11]:
stats['GameHome']=stats['Game_ID'].astype(str)+'_'+stats['Team'].astype(str)
stats['GameAway']=stats['Game_ID'].astype(str)+'_'+stats['Team'].astype(str)

In [12]:
stats['Home Team']=stats.Team.copy()
stats['Away Team']=stats.Team.copy()

In [13]:
stats['SoF']=(stats.Shots+0.1)-(stats.SoT+0.1)
stats['dominance']=((stats.Passes>0)|(stats.Shots>0)).astype('int')
stats['duration']=stats.End_minutes-stats.Start_minutes
stats['played']=stats.dominance*stats['duration']
stats['Y30']=((stats.Y>30)&(stats['Shots']==0)&(stats['Passes']==0)).astype('int')
stats['X105']=((stats.X==105)&(stats['Shots']==0)&(stats['Passes']==0)).astype('int')
stats['X92']=((stats.X>=92)&(stats.Y>30)&(stats['Shots']==0)&(stats['Passes']==0)).astype('int')
stats['X13']=((stats.X<=13)&(stats.Y<=337.1)&(stats['Shots']==0)&(stats['Passes']==0)).astype('int')

# Home & Away Team Features

In [14]:
HomeSum=stats[['GameHome', 'Y30','X92','X105','X13',
                 'Shots', 
                 'SoT',
                 'Accurate passes',
             'dominance',
                'Inaccurate passes',
                 'SoF',
                 'duration',
               'played'
                ]].groupby(['GameHome']).sum().add_suffix('Sum').reset_index()
AwaySum=stats[['GameAway','Y30','X92','X105','X13',
                 'Shots', 
                 'SoT',
                 'Accurate passes',
               'dominance',
                'Inaccurate passes',
                 'SoF',
                 'duration',
              'played'
                ]].groupby(['GameAway']).sum().add_suffix('Sum').reset_index()

In [15]:
HomeSkew=stats[['GameHome', 'Y30','X92','X105','X13',
                 'Shots', 
                 'SoT',
                 'Accurate passes',
             'dominance',
                'Inaccurate passes',
                 'SoF',
                 'duration',
               'played'
                ]].groupby(['GameHome']).skew().add_suffix('Skew').reset_index()
AwaySkew=stats[['GameAway','Y30','X92','X105','X13',
                 'Shots', 
                 'SoT',
                 'Accurate passes',
               'dominance',
                'Inaccurate passes',
                 'SoF',
                 'duration',
              'played'
                ]].groupby(['GameAway']).skew().add_suffix('Skew').reset_index()

In [16]:
HomeStd=stats[['GameHome','Y30','X92','X105','X13',
                'Shots', 
                 'SoT',
                 'Accurate passes',
              'dominance',
                'Inaccurate passes',
                 'SoF',
                 'duration',
                'played'
                ]].groupby(['GameHome']).std().add_suffix('Std').reset_index()
AwayStd=stats[['GameAway','Y30','X92','X105','X13',
                 'Shots', 
                 'SoT',
                 'Accurate passes',
                 'dominance',
                'Inaccurate passes',
                 'SoF',
                 'duration',
                 'played'
                ]].groupby(['GameAway']).std().add_suffix('Std').reset_index()
                
HomeStd=pd.merge(HomeStd,HomeSum,on='GameHome',how='left')
AwayStd=pd.merge(AwayStd,AwaySum,on='GameAway',how='left')

HomeStd=pd.merge(HomeStd,HomeSkew,on='GameHome',how='left')
AwayStd=pd.merge(AwayStd,AwaySkew,on='GameAway',how='left')


In [17]:
HomeStats=stats[['GameHome','Y30','X92','X105','X13',
                'Shots', 
                 'SoT',
                 'Accurate passes',
              'dominance',
                'Inaccurate passes',
                 'SoF',
                 'duration',
                'played'
                ]].groupby(['GameHome']).mean().reset_index()
AwayStats=stats[['GameAway','Y30','X92','X105','X13',
                 'Shots', 
                 'SoT',
                 'Accurate passes',
                 'dominance',
                'Inaccurate passes',
                 'SoF',
                 'duration',
                 'played'
                ]].groupby(['GameAway']).mean().reset_index()

HomeStats=pd.merge(HomeStats,HomeStd,on='GameHome',how='left')
AwayStats=pd.merge(AwayStats,AwayStd,on='GameAway',how='left')
stat_cols=list(HomeStats.columns[1:])

In [18]:
HomeStats['Home Team']=(HomeStats.GameHome.str.split('_',2,expand=True)[2]).astype('int')
AwayStats['Away Team']=(AwayStats.GameAway.str.split('_',2,expand=True)[2]).astype('int')

In [19]:
TeamHome=HomeStats.groupby('Home Team').mean().reset_index()
TeamAway=AwayStats.groupby('Away Team').mean().reset_index()

In [20]:
TeamHome=TeamHome.add_suffix(' mean')
TeamAway=TeamAway.add_suffix(' mean')
mean_cols=list(TeamHome.columns[1:])

In [21]:
TeamHome=TeamHome.rename({'Home Team mean':'Home Team'},axis=1)
TeamAway=TeamAway.rename({'Away Team mean':'Away Team'},axis=1)

In [22]:
HomeManager=stats[['GameHome','Manager','no_of_managers','no_of_teams']].drop_duplicates('GameHome')
AwayManager=stats[['GameAway','Manager','no_of_managers','no_of_teams']].drop_duplicates('GameAway')

In [23]:
stat_cols=stat_cols+['no_of_managers','no_of_teams']

In [24]:
tdf['Score']=np.nan

In [25]:
df=df.append(tdf[df.columns])

In [26]:
df['Home Team'] = teamEncoder.transform(df['Home Team']).astype('int')
df['Away Team'] = teamEncoder.transform(df['Away Team']).astype('int')

In [27]:
GameHome1=df.copy()
GameHome2=df.copy()
GameHome1['GameHome']=GameHome1['Game_ID'].astype(str)+'_'+GameHome1['Home Team'].astype(str)
GameHome2['GameHome']=GameHome2['Game_ID'].astype(str)+'_'+GameHome2['Away Team'].astype(str)
GameHome=GameHome1[['GameHome','Season']].append(GameHome2[['GameHome','Season']])
GameHome=GameHome.drop_duplicates(['GameHome','Season'])

In [28]:
df['Min Team'] = df[['Home Team','Away Team']].min(axis=1)
df['Max Team'] = df[['Home Team','Away Team']].max(axis=1)

In [29]:
df['H']=df['Home Team'].copy()
df['A']=df['Away Team'].copy()
df['MinisHome']=(df['H']==df['Min Team']).astype('int')
df['Home Team']=df['Min Team'].copy()
df['Away Team'] = df['Max Team'].copy()

In [30]:
df.drop(['Min Team','Max Team'],axis=1,inplace=True)

In [31]:
Season=df.copy()[['Home Team','Away Team','Score']]
Season=Season[Season.Score.notna()]
Season['Home Points'] = Season['Score'].replace({'Away win':0,'Draw':1,'Home Win':3})
Season['Away Points'] = Season['Score'].replace({'Home Win':0,'Draw':1,'Away win':3})
Season['Home Matches'] = Season.groupby(['Home Team'])['Home Team'].transform('count')
Season['Away Matches'] = Season.groupby(['Away Team'])['Away Team'].transform('count')
Season['Home Points'] = Season.groupby(['Home Team'])['Home Points'].transform('sum')
Season['Away Points'] = Season.groupby(['Away Team'])['Away Points'].transform('sum')

In [32]:
HomeSeason = Season[['Home Team','Home Points','Home Matches']].drop_duplicates(['Home Team'])

In [33]:
AwaySeason = Season[['Away Team','Away Points','Away Matches']].drop_duplicates(['Away Team'])

In [34]:
others=AwaySeason[~AwaySeason['Away Team'].isin(HomeSeason['Home Team'])]
others=pd.DataFrame(others.values,columns=list(HomeSeason.columns))
others[['Home Points','Home Matches']]=0
HomeSeason=HomeSeason.append(others)

In [35]:
Seasons = pd.merge(HomeSeason.rename({'Home Team':'Team'},axis=1),AwaySeason.rename({'Away Team':'Team'},axis=1), on=['Team'],how='left')
Seasons.fillna(0,inplace=True)
print(Seasons.describe())
Scols = list(Seasons.columns[1:])
Seasons['Points'] = Seasons['Home Points'] + Seasons['Away Points']
Seasons['Matches'] = Seasons['Home Matches'] + Seasons['Away Matches']
Seasons['Av Points'] = Seasons['Points'] / Seasons['Matches']
Seasons = Seasons.drop(Scols,axis=1)
Scols = list(Seasons.columns[2:])
Scols=['Av Points']
Seasons.drop(['Points','Matches'],axis=1,inplace=True)

            Team  Home Points  Home Matches  Away Points  Away Matches
count  19.000000    19.000000     19.000000    19.000000     19.000000
mean    9.631579    33.000000     21.684211    24.947368     21.684211
std     6.404951    24.147694     16.052181    13.554309     13.059916
min     0.000000     0.000000      0.000000     0.000000      0.000000
25%     4.500000    15.000000      7.500000    15.500000     15.000000
50%     9.000000    36.000000     22.000000    25.000000     23.000000
75%    15.000000    53.000000     34.500000    32.000000     29.000000
max    20.000000    78.000000     50.000000    54.000000     43.000000


In [36]:
df['GameHome']=df['Game_ID'].astype(str)+'_'+df['Home Team'].astype(str)
df['GameAway']=df['Game_ID'].astype(str)+'_'+df['Away Team'].astype(str)

In [37]:
df['Score'] = df['Score'].replace({'Draw':1,'Away win':0, 'Home Win':2})
df.Score[df.MinisHome==0]=(df.Score-2).abs()

In [38]:
df.Score.fillna(4,inplace=True)
df.Score = df.Score.astype('int')

In [39]:
df['Date'] = pd.to_datetime(df['Date'])
df['year'] = df['Date'].dt.year
df['year']=df['year']-df['year'].min() + 1
df['month'] = df['Date'].dt.month
df['period'] = df['month']*df['year']
df['mid'] = (df.period/6).astype('int')
df['timestamp'] = (df['Date'] - df['Date'].min()).astype('str')
df['timestamp'] = df['timestamp'].str[:-5].astype('int')

In [40]:
df=df.sort_values('timestamp')

In [41]:
df['Result']=df['Score']
df=pd.get_dummies(df,columns=['Result'])

In [42]:
target_cols=['Score','Result_0','Result_1','Result_2','Result_4']
result_cols=target_cols[1:4]
target = 'Score'

In [43]:
df['HomeAway']=df['Home Team'].astype(str) + '_' + df['Away Team'].astype(str)

In [44]:
drop_cols=[
    'Date',
    'Match_ID',
    'Game_ID','GameHome','GameAway',
    'mid',
   'H',
    'A',
    'MinisHome'
] + target_cols

In [45]:
GameAway=GameHome.rename({'GameHome':'GameAway'},axis=1)

In [46]:
HomeStats=pd.merge(HomeStats,GameHome,on='GameHome',how='left')
AwayStats=pd.merge(AwayStats,GameAway,on='GameAway',how='left')

In [47]:
for col in stat_cols[:-2]:
  HomeStats[col+'Season']=HomeStats.groupby(['Home Team','Season'])[col].transform('mean')
  AwayStats[col+'Season']=AwayStats.groupby(['Away Team','Season'])[col].transform('mean')
  stat_cols=stat_cols+[col+'Season']


In [48]:
HomeStats=HomeStats.drop(['Season','Home Team'],axis=1)
AwayStats=AwayStats.drop(['Season','Away Team'],axis=1)

In [49]:
df=pd.merge(df,HomeManager,on='GameHome',how='left')
df=pd.merge(df,AwayManager,on='GameAway',how='left')
df=pd.merge(df,HomeStats,on='GameHome',how='left')
df=pd.merge(df,AwayStats,on='GameAway',how='left')
for col in stat_cols:
    df[col+'_div']=(df[col+'_x']+0.1)/(df[col+'_y']+0.1)

In [50]:
df=df.sort_values('period')
cols = list(df.drop(drop_cols+['HomeAway'],axis=1).columns)
for col in cols:
  df[col+'prev']=df.groupby(['HomeAway'])[col].shift(1)
  df[col+'prevDiff']=(df[col]+1)/(df[col+'prev']+1)
for col in cols:
  df[col+'prev2']=df.groupby(['HomeAway'])[col].shift(2)
  df[col+'prev2Diff']=(df[col]+1)/(df[col+'prev2']+1)
for col in cols:
  df[col+'prev3']=df.groupby(['HomeAway'])[col].shift(3)
  df[col+'prev3Diff']=(df[col]+1)/(df[col+'prev3']+1)

In [51]:
le=LabelEncoder()
for col in ['HomeAway']:
    df[col]=le.fit_transform(df[col].astype(str)).astype('int')

In [52]:
cols = list(df.drop(drop_cols,axis=1).columns)
cols=cols[::3]

In [53]:
tdf=df[df[target]==4]
df=df[df[target]<4]
model=CatBoostClassifier(silent=True,learning_rate=0.1,n_estimators=1000)
pred_cols=['pred_0','pred_1','pred_2']
model.fit(df[cols],df[target])
tdf[result_cols]=model.predict_proba(tdf[cols])


In [54]:
tdf['Away win']=tdf['Result_0'].copy()
tdf['Home Win']=tdf['Result_2'].copy()
tdf['Draw']=tdf['Result_1'].copy()
tdf['Away win'][tdf.MinisHome==0]=tdf['Result_2']
tdf['Home Win'][tdf.MinisHome==0]=tdf['Result_0']


In [55]:
sub=tdf[ss.columns]
sub=pd.merge(ss[['Game_ID']],sub,on='Game_ID',how='left')
sub.to_csv('playbook.csv',index=None)
sub

,Game_ID,Away win,Draw,Home Win
0,ID_8518U587,0.118376,0.639192,0.242432
1,ID_H49BIKG7,0.001514,0.041572,0.956914
2,ID_PO6SP4VA,0.931302,0.061964,0.006734
3,ID_MZRCNBAQ,0.707788,0.280632,0.011580
4,ID_CV9VOLIU,0.001217,0.006004,0.992779
...,...,...,...,...
229,ID_QKQ00O16,0.308634,0.665170,0.026196
230,ID_172V2IXW,0.009399,0.840586,0.150015
231,ID_2ZK5TE39,0.245680,0.741480,0.012840
232,ID_IS54QGW9,0.007075,0.745471,0.247454


In [56]:
sorted(zip(map(lambda x: round(x, 4), model.feature_importances_), cols), reverse=True)[:10]

[(21.418, 'durationStd_div'),
 (5.8569, 'durationStd_x'),
 (5.4013, 'durationStd_y'),
 (4.3561, 'durationSkew_x'),
 (4.0223, 'durationSkew_y'),
 (3.6249, 'duration_div'),
 (2.2485, 'durationSkew_div'),
 (1.4451, 'duration_x'),
 (1.3307, 'dominanceSum_x'),
 (1.3075, 'durationSumSeason_y')]